## Predicting Hospital Readmissions Using Integrated Patient, Clinical, and Socioeconomic Data
 
1.2.1	🎯 Project Objective:
To develop a predictive model for 30-day hospital readmission risk by merging and cleaning patient demographics, clinical encounter data, and socioeconomic data. The goal is to help hospitals reduce readmissions, improve patient outcomes, and reduce costs.
### J. Casey Brookshier
### 7/21/2025

## "Hospital Quality Forecasting: Data-Driven Insights into Readmission Penalties"
Recommended Workflow: Clean First, Then Integrate
## In short: Clean → Standardize → Aggregate → Integrate → Analyze


In [ ]:
# Hospital Readmission Risk Forecasting

## Objective
Predict hospital-level 30-day readmission risk using publicly available
CMS readmission metrics, healthcare-associated infection indicators,
and socioeconomic deprivation (ADI).

## Business Value
• Identify facilities at risk of CMS readmission penalties  
• Support targeted quality improvement initiatives  
• Enable data-informed policy and administrative decisions


In [ ]:
hospital_readmission_forecasting/
│
├── data/
│   ├── FY_2025_Hospital_Readmissions_Reduction_Program_Hospital.csv
│   ├── Healthcare_Associated_Infections-Hospital.csv
│   ├── CO_2023_ADI_9 Digit Zip Code_v4_0_1.csv
│   └── hospital_readmissions_analytic_table.csv  
│
├── artifacts/
│   ├── random_forest_model.pkl
│   ├── feature_names.pkl
│   └── imputer.pkl
│
├── src/
│   ├── prepare_data.py        
│   └── train_readmissions_model.py
│
├── requirements.txt
└── README.md




In [ ]:
# ============================================================
# Hospital Readmission Forecasting – Data Preparation Script
# ============================================================
# Author: J. Casey Brookshier
# Purpose: Build analytic dataset for readmission modeling
# Inputs: Raw CMS Readmissions, Infections, ADI files
# Output: hospital_readmissions_analytic_table.csv
# ============================================================

import pandas as pd
import numpy as np
from pathlib import Path

# ============================================================
# CONFIG (RELATIVE PATHS)
# ============================================================
# find project root dynamically (relative paths for location independent functionality)
PROJECT_ROOT = Path(__file__).resolve().parents[1]

DATA_DIR = PROJECT_ROOT / "data"
# define where merged dataset to be written
OUTPUT_FILE = DATA_DIR / "hospital_readmissions_analytic_table.csv"

# define filepaths for raw data sources
READMISSIONS_FILE = DATA_DIR / "FY_2025_Hospital_Readmissions_Reduction_Program_Hospital.csv"
INFECTIONS_FILE   = DATA_DIR / "Healthcare_Associated_Infections-Hospital.csv"
ADI_FILE          = DATA_DIR / "CO_2023_ADI_9 Digit Zip Code_v4_0_1.csv"

# Define readmission columns of interest, centralize metric control
READMISSION_METRICS = [
    "Excess Readmission Ratio",
    "Predicted Readmission Rate",
    "Expected Readmission Rate",
]

# log confirmation of start of pipeline execution
print("🔧 Rebuilding analytic dataset from raw sources")

# ============================================================
# LOAD DATA -> read CSV's into pandas DF
# ============================================================

readm_df = pd.read_csv(READMISSIONS_FILE)
infect_df = pd.read_csv(INFECTIONS_FILE)
adi_df    = pd.read_csv(ADI_FILE)

# ============================================================
# CANONICAL KEY NORMALIZATION -> force Facility ID into string
# ============================================================

readm_df["Facility ID"] = readm_df["Facility ID"].astype(str)
infect_df["Facility ID"] = infect_df["Facility ID"].astype(str)
# normalize ZIP into 5 digit strings for reliable ZIP/ADI join
infect_df["ZIP Code"] = infect_df["ZIP Code"].astype(str).str.zfill(5)

# ============================================================
# CLEAN READMISSIONS DATA
# ============================================================
# create working copy
readm = readm_df.copy()

# convert verbose CMS names into clean feature identifiers
readm["measure_code"] = (
    readm["Measure Name"]
    .str.replace("READM-30-", "", regex=False)
    .str.replace("-HRRP", "", regex=False)
    .str.lower()
)
# convert readmission metric to numeric, invalid values to NaN
for col in READMISSION_METRICS:
    readm[col] = pd.to_numeric(readm[col], errors="coerce")
    
# remove hospital w/o valid readmission signal
readm = readm.dropna(subset=["Excess Readmission Ratio"])

# reshape data long ->wide, one facility one row, one condition one column
readm_pivot = readm.pivot_table(
    index=["Facility ID", "Facility Name", "State"],
    columns="measure_code",
    values=READMISSION_METRICS
)

# flatten pandas' multi-index columns, create ML-ready feature names
readm_pivot.columns = [
    f"{metric.replace(' ', '_').lower()}__{measure}"
    for metric, measure in readm_pivot.columns
]
# change index fields back into columns
readm_pivot = readm_pivot.reset_index()

# check for facility ID pipeline corruption
assert "Facility ID" in readm_pivot.columns, "Facility ID missing after pivot"

# ============================================================
# CLEAN INFECTIONS DATA
# ============================================================
# working copy
infect = infect_df.copy()

# infection scores to numeric, remove invalid rows
infect["Score"] = pd.to_numeric(infect["Score"], errors="coerce")
infect = infect.dropna(subset=["Score"])

# standardize measure names into clean column identifiers
infect["measure_clean"] = (
    infect["Measure Name"]
    .str.replace("[^A-Za-z0-9]+", "_", regex=True)
    .str.lower()
)

# aggregate infection metrics per hospital
infect_pivot = infect.pivot_table(
    index="Facility ID",
    columns="measure_clean",
    values="Score",
    aggfunc="mean"
).add_prefix("infection__")

# prep for merge
infect_pivot = infect_pivot.reset_index()

# ============================================================
# CLEAN ADI DATA 
# ============================================================
# improve column readability
adi = adi_df.rename(columns={
    "ZIP_4": "zip",
    "ADI_NATRANK": "adi_national",
    "ADI_STATERNK": "adi_state",
})

# Keep only required columns to avoid dtype pollution
adi = adi[["zip", "adi_national", "adi_state"]]

# norm ZIP to 5 value string
adi["zip"] = adi["zip"].astype(str).str.zfill(5)

# adi values numeric/valid
adi["adi_national"] = pd.to_numeric(adi["adi_national"], errors="coerce")
adi["adi_state"] = pd.to_numeric(adi["adi_state"], errors="coerce")
adi = adi.dropna(subset=["adi_national", "adi_state"])
adi = adi.astype({
    "adi_national": "float64",
    "adi_state": "float64",
})
# aggregate 9 digit zip to 5 digit
adi_zip = (
    adi.groupby("zip", as_index=False)
       .mean(numeric_only=True)
)
# verify aggregation produces numeric
assert adi_zip[["adi_national", "adi_state"]].apply(
    lambda s: np.issubdtype(s.dtype, np.number)
).all(), "ADI aggregation produced non-numeric output"

# ============================================================
# FACILITY → ZIP → ADI BRIDGE
# ============================================================
# build facility to ZIP lookup table
facility_zip = (
    infect_df[["Facility ID", "ZIP Code"]]
    .drop_duplicates()
    .rename(columns={"ZIP Code": "zip"})
)
facility_zip["Facility ID"] = facility_zip["Facility ID"].astype(str)
facility_zip["zip"] = facility_zip["zip"].astype(str).str.zfill(5)

# attach SES context by facility
facility_adi = facility_zip.merge(
    adi_zip,
    on="zip",
    how="left"
)

assert "Facility ID" in facility_adi.columns, "Facility ID missing in facility_adi"

# ============================================================
# FINAL ANALYTIC TABLE -> readmissions + infections +SES data
# ============================================================
# create merged ML ready DF
final_df = (
    readm_pivot
    .merge(infect_pivot, on="Facility ID", how="left")
    .merge(
        facility_adi[["Facility ID", "adi_national", "adi_state"]],
        on="Facility ID",
        how="left"
    )
)

# ============================================================
# TARGET CONSTRUCTION
# ============================================================
# identify excess readmission metrics
excess_cols = [
    c for c in final_df.columns
    if c.startswith("excess_readmission_ratio__")
]
# create model target, AVE excess readmission
final_df["composite_readmission_score"] = final_df[excess_cols].mean(axis=1)

# ============================================================
# FINAL VALIDATION -> raise error if key columns missing
# ============================================================

REQUIRED_COLUMNS = {
    "Facility ID",
    "Facility Name",
    "State",
    "composite_readmission_score",
}
missing = REQUIRED_COLUMNS - set(final_df.columns)
if missing:
    raise ValueError(f"Missing required columns in final dataset: {missing}")

# ============================================================
# SAVE OUTPUT, write analytic dataset to disk, report size
# ============================================================

final_df.to_csv(OUTPUT_FILE, index=False)

print(f"✅ Analytic dataset saved: {OUTPUT_FILE.resolve()}")
print(f"📦 Final shape: {final_df.shape}")


In [ ]:
# ============================================================
# train_readmissions_model.py
# ============================================================
# Purpose:
#   Train and evaluate readmission risk models using the
#   prepared analytic dataset.
# ============================================================

import pandas as pd
import numpy as np
from pathlib import Path
# create train/test datasets, perform k-fold cross-validation
from sklearn.model_selection import train_test_split, cross_val_score
# create baseline linear regression model, random forest model
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
# model evaluation metrics
from sklearn.metrics import mean_squared_error, r2_score
# handle missing values -> replace with column means
from sklearn.impute import SimpleImputer
# serialize python objects to disk
import pickle

# ============================================================
# PATHS -> # find project root dynamically 
# (relative paths for location independent functionality)
# ============================================================

PROJECT_ROOT = Path(__file__).resolve().parents[1
# define where input data is and where trained models saved
DATA_DIR = PROJECT_ROOT / "data"
ARTIFACT_DIR = PROJECT_ROOT / "artifacts"
# define path to dataset created by prev script
DATA_FILE = DATA_DIR / "hospital_readmissions_analytic_table.csv"

# ============================================================
# LOAD DATA -> load merged data set into df
# ============================================================

df = pd.read_csv(DATA_FILE)

# define relevant columns for effective modeling
REQUIRED_COLUMNS = {
    "Facility ID",
    "Facility Name",
    "State",
    "composite_readmission_score",
}

missing = REQUIRED_COLUMNS - set(df.columns)
if missing:
    raise ValueError(f"Required column(s) missing: {missing}")

print(f"✅ Loaded dataset: {df.shape}")

# ============================================================
# MODEL PREP -> define prediction target (dependent variable)
# ============================================================

TARGET = "composite_readmission_score"

# create feature matrix x, minus non-pred identifiers, target
X = df.drop(columns=[
    "Facility ID",
    "Facility Name",
    "State",
    TARGET,
])

# extract target vector
y = df[TARGET]

# remove missing columns
X = X.dropna(axis=1, how="all")

# implement mean value imputer for missing values
imputer = SimpleImputer(strategy="mean")

# fit imputer on full dataset, wrap output into DF w/ proper labels
X = pd.DataFrame(
    imputer.fit_transform(X),
    columns=X.columns,
    index=X.index,
)

# ============================================================
# TRAIN / TEST -> 80/20 train/test split
# ============================================================

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# instantiate LR model, fit to train data
lr = LinearRegression()
lr.fit(X_train, y_train)

# 100 trees, reproducible randomness, use all CPU cores
rf = RandomForestRegressor(
    n_estimators=100,
    random_state=42,
    n_jobs=-1,
)
rf.fit(X_train, y_train) #train random forest

# ============================================================
# EVALUATION -> function to predict on test data
# compute RMSE, R^2, return metrics as dictionary
# ============================================================

def evaluate(model):
    preds = model.predict(X_test)
    return {
        "RMSE": np.sqrt(mean_squared_error(y_test, preds)),
        "R2": r2_score(y_test, preds),
    }

# LR & random forest eval on test set, print results
print("\n📊 Model Performance")
print("Linear Regression:", evaluate(lr))
print("Random Forest:", evaluate(rf))

# perform 5-fold cross validation on entire dataset
cv_rmse = np.sqrt(
    -cross_val_score(
        rf,
        X,
        y,
        cv=5,
        scoring="neg_mean_squared_error",
    )
)

# print ave RMSE, variability of model performance
print("\n📈 Random Forest CV RMSE")
print("Mean:", round(cv_rmse.mean(), 4), "Std:", round(cv_rmse.std(), 4))

# ============================================================
# SAVE ARTIFACTS -> create artifacts/dir if it doesn't exist
# save trained RF mod, feature names, fitted imputer for 
# consistent missing values
# ============================================================

ARTIFACT_DIR.mkdir(exist_ok=True)

with open(ARTIFACT_DIR / "random_forest_model.pkl", "wb") as f:
    pickle.dump(rf, f)

with open(ARTIFACT_DIR / "feature_names.pkl", "wb") as f:
    pickle.dump(list(X.columns), f)

with open(ARTIFACT_DIR / "imputer.pkl", "wb") as f:
    pickle.dump(imputer, f)

print("\n✅ Model artifacts saved")
